In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/hotel_data_with_embeddings.pkl')
df.head()

,hotel_name,hotel_description,hotel_description_summary,rating_value,review_count,review_title,review_text,rate,tripdate,price_range,...,full_review,full_address,description_processed,full_review_processed,address_processed,price_processed,embedding_descriptions,embedding_addresses,embedding_prices,embedding_full_review
0,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Romance Istanbul Hotel offers 39 elegantly fur...,5.0,4023.0,"An exceptional boutique hotel, great value for...","This boutique hotel exceeded all expectations,...",5.0,2020-02,"affordable, budget, economical",...,"An exceptional boutique hotel, great value for...","Hudavendigar Cd. No:5 Sirkeci, Istanbul, Turkiye",romanc istanbul hotel 39 room elegantli furnis...,except boutiqu hotel great valu money boutiqu ...,hudavendigar cd sirkeci istanbul turkiy,afford budget econom,"[-0.72195345, 1.079411, -3.1429782, -0.9376375...","[-0.57478076, 0.008897147, -3.1645122, -0.2190...","[0.41714233, 0.93826044, -2.9774604, -0.179409...","[-1.4592323, 1.7448821, -3.8982518, 0.91076475..."
1,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Romance Istanbul Hotel offers 39 elegantly fur...,5.0,4023.0,You can’t get better than this.,An exceptional stay that exceeded all expectat...,5.0,2021-03,"affordable, budget, economical",...,You can’t get better than this.: An exceptiona...,"Hudavendigar Cd. No:5 Sirkeci, Istanbul, Turkiye",romanc istanbul hotel 39 room elegantli furnis...,get better except stay exceed expect hotel del...,hudavendigar cd sirkeci istanbul turkiy,afford budget econom,"[-0.72195345, 1.079411, -3.1429782, -0.9376375...","[-0.57478076, 0.008897147, -3.1645122, -0.2190...","[0.41714233, 0.93826044, -2.9774604, -0.179409...","[-0.7813845, 1.2947574, -3.6422498, 0.23717952..."
2,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Romance Istanbul Hotel offers 39 elegantly fur...,5.0,4023.0,Exceeds all expectations,This hotel truly goes above and beyond in ever...,5.0,2021-03,"affordable, budget, economical",...,Exceeds all expectations: This hotel truly goe...,"Hudavendigar Cd. No:5 Sirkeci, Istanbul, Turkiye",romanc istanbul hotel 39 room elegantli furnis...,exce expect hotel truli goe beyond everi aspec...,hudavendigar cd sirkeci istanbul turkiy,afford budget econom,"[-0.72195345, 1.079411, -3.1429782, -0.9376375...","[-0.57478076, 0.008897147, -3.1645122, -0.2190...","[0.41714233, 0.93826044, -2.9774604, -0.179409...","[-0.75293285, 1.4301245, -3.5443554, 0.1793581..."
3,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Romance Istanbul Hotel offers 39 elegantly fur...,5.0,4023.0,"Great Location, Fantastic Accommodations",This hotel offers a superb experience with its...,5.0,2021-08,"affordable, budget, economical",...,"Great Location, Fantastic Accommodations: This...","Hudavendigar Cd. No:5 Sirkeci, Istanbul, Turkiye",romanc istanbul hotel 39 room elegantli furnis...,great locat fantast accommod hotel offer super...,hudavendigar cd sirkeci istanbul turkiy,afford budget econom,"[-0.72195345, 1.079411, -3.1429782, -0.9376375...","[-0.57478076, 0.008897147, -3.1645122, -0.2190...","[0.41714233, 0.93826044, -2.9774604, -0.179409...","[-1.6934611, 1.8118676, -4.088441, -0.30559927..."
4,Romance Istanbul Hotel,Romance Istanbul Hotel has 39 rooms.Every room...,Romance Istanbul Hotel offers 39 elegantly fur...,5.0,4023.0,Perfection. It is all in the details.,Impeccable attention to detail elevates the ex...,5.0,2021-06,"affordable, budget, economical",...,Perfection. It is all in the details.: Impecca...,"Hudavendigar Cd. No:5 Sirkeci, Istanbul, Turkiye",romanc istanbul hotel 39 room elegantli furnis...,perfect detail impecc attent detail elev exper...,hudavendigar cd sirkeci istanbul turkiy,afford budget econom,"[-0.72195345, 1.079411, -3.1429782, -0.9376375...","[-0.57478076, 0.008897147, -3.1645122, -0.2190...","[0.41714233, 0.93826044, -2.9774604, -0.179409...","[-0.2489

In [ ]:
import numpy as np
embeddings_addresses_array = np.vstack(df['embedding_addresses'])
embeddings_prices_array = np.vstack(df['embedding_prices'])
embeddings_full_review_array = np.vstack(df['embedding_full_review'])
embeddings_descriptions_array = np.vstack(df['embedding_descriptions'])

print(embeddings_addresses_array.shape)
print(embeddings_prices_array.shape)
print(embeddings_full_review_array.shape)
print(embeddings_descriptions_array.shape)

(5997, 768)
(5997, 768)
(5997, 768)
(5997, 768)


In [ ]:
combined_embeddings = np.hstack([
    embeddings_addresses_array,
    embeddings_prices_array,
    embeddings_full_review_array,
    embeddings_descriptions_array
])

print("Shape of combined_embeddings:", combined_embeddings.shape)

Shape of combined_embeddings: (5997, 3072)


In [ ]:
!pip install sentence_transformers

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
import nltk

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Tokenize and convert to lowercase
    tokens = word_tokenize(text.lower())

    # Keep only alphanumeric tokens (including alphabetic and numeric characters)
    tokens = [word for word in tokens if word.isalnum()]

    # Remove stop words and apply stemming
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5" , trust_remote_code=True)

def embed(text):
    embeddings = model.encode(text, convert_to_tensor=True)
    embeddings = embeddings.to(device)
    return embeddings

In [ ]:
def embed_query(query):
    query_preprocessing = preprocess_text(query)
    query_embedding = embed([query_preprocessing]).cpu().numpy()[0]
    query_addresses_embedding = embed([query_preprocessing]).cpu().numpy()[0]
    query_prices_embedding = embed([query_preprocessing]).cpu().numpy()[0]
    query_full_review_embedding = embed([query_preprocessing]).cpu().numpy()[0]
    query_descriptions_embedding = embed([query_preprocessing]).cpu().numpy()[0]
    query_combined_embedding = np.concatenate([
        query_addresses_embedding,
        query_prices_embedding,
        query_full_review_embedding,
        query_descriptions_embedding
    ])
    query_combined_embedding = query_combined_embedding.reshape(1, -1)
    return query_combined_embedding, query_embedding

In [ ]:
query = "A good hotel with a great view on New York"
query_combined_embedding, query_single_embedding = embed_query(query)
print(query_combined_embedding.shape)
print(query_single_embedding.shape)

(1, 3072)
(768,)


In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install rank_bm25

In [ ]:
import faiss
from rank_bm25 import BM25Okapi
import numpy as np

def faiss_bm25(df, query, k1=1.5, b=0.75, combined_embeddings=None, query_embedding=None, k=200, nlist=50, nprobe=100, bm25_weight=0.3, faiss_weight=0.7):
    """
    Perform a hybrid search using both BM25 and Faiss with weighted scores.

    Parameters:
    - df: pandas DataFrame containing tokenized text columns and embeddings
    - query: string representing the search query
    - k1: BM25 parameter k1
    - b: BM25 parameter b
    - combined_embeddings: numpy array with combined embeddings for Faiss
    - query_embedding: numpy array with query embedding for Faiss
    - k: number of top results to retrieve from Faiss
    - nlist: number of clusters for the Faiss index
    - nprobe: number of clusters to search in Faiss
    - bm25_weight: weight for BM25 scores
    - faiss_weight: weight for Faiss scores

    Returns:
    - metadata: String containing formatted metadata for the top k results
    """

    # Combine all tokenized fields for BM25 search
    combined_columns = [
        description + " " + full_review + " " + address + " " + price
        for description, full_review, address, price in zip(
            df['description_processed'], df['full_review_processed'], df['address_processed'], df['price_processed']
        )
    ]

    # Initialize and run BM25 model
    bm25 = BM25Okapi(combined_columns, k1=k1, b=b)
    query_preprocessing = preprocess_text(query)
    bm25_scores = bm25.get_scores(query_preprocessing)

    # Initialize and train Faiss index
    combined_dimension = combined_embeddings.shape[1]
    quantizer = faiss.IndexFlatL2(combined_dimension)
    index = faiss.IndexIVFFlat(quantizer, combined_dimension, nlist)

    # Train the index
    index.train(combined_embeddings)
    assert index.is_trained, "Faiss index failed to train properly."

    # Add embeddings to the index
    index.add(combined_embeddings)
    index.nprobe = nprobe  # Set nprobe to control the number of clusters to search

    # Perform Faiss search
    query_combined_embedding = query_embedding.reshape(1, -1)
    _, faiss_indices = index.search(query_combined_embedding, k)

    # Combine BM25 and Faiss results
    faiss_scores = np.zeros(len(df))
    for idx in faiss_indices[0]:
        faiss_scores[idx] = 1  # Assign a dummy score of 1 for Faiss results

    # Normalize scores
    faiss_scores = (faiss_scores - faiss_scores.min()) / (faiss_scores.max() - faiss_scores.min())

    # Weighted combination of BM25 and Faiss scores
    combined_scores = bm25_weight * np.array(bm25_scores) + faiss_weight * faiss_scores

    # Get top k results based on combined scores
    top_k_indices = np.argsort(-combined_scores)[:k]

    # Group reviews by hotel name
    hotel_reviews = {}
    for idx in top_k_indices:
        row = df.iloc[idx]
        hotel_name = row['hotel_name']
        if hotel_name not in hotel_reviews:
            hotel_reviews[hotel_name] = {
                'hotel_description_summary': row['hotel_description_summary'],
                'price_range': row['price_range'],
                'trip_date': row['tripdate'],
                'hotel_url': row['hotel_url'],
                'rating_value': row['rating_value'],
                'review_count': row['review_count'],
                'country': row['country'],
                'city': row['locality'],
                'street_address': row['street_address'],
                'hotel_image': row['hotel_image'],
                'reviews': []
            }
        hotel_reviews[hotel_name]['reviews'].append({
            'review_title': row['review_title'],
            'review_text': row['review_text'],
            'rating': row['rate'],
        })

    # Prepare formatted metadata output
    metadata = []
    for hotel_name, details in hotel_reviews.items():
        metadata.append(
            f"Hotel Name: {hotel_name}\n"
            f"Description: {details['hotel_description_summary']}\n"
            f"Price Range: {details['price_range']}\n"
            f"Trip Date: {details['trip_date']}\n"
            f"Hotel URL: {details['hotel_url']}\n"
            f"Rating Value: {details['rating_value']}\n"
            f"Review Count: {details['review_count']}\n"
            f"Country: {details['country']}\n"
            f"City: {details['city']}\n"
            f"Street Address: {details['street_address']}\n"
            f"Hotel Image: {details['hotel_image']}\n"
        )
        for review in details['reviews']:
            metadata.append(
                f"\nReview Title: {review['review_title']}\n"
                f"Review Text: {review['review_text']}\n"
                f"Rating: {review['rating']}\n"
            )
        metadata.append("-" * 80)

    return "\n".join(metadata)

In [ ]:
metadata = faiss_bm25(df, query, combined_embeddings=combined_embeddings, query_embedding=query_combined_embedding, k=10)

In [ ]:
!pip install openai==0.27.7
!pip install gradio

In [ ]:
from google.colab import userdata
import openai
openai.api_key = '#'

In [ ]:
import openai
import markdown
from IPython.display import display, HTML

def search(query):
    return faiss_bm25(df, query, combined_embeddings=combined_embeddings, query_embedding=query_combined_embedding, k=200)

def generate_answer(query):
    context = search(query)
    prompt = f"""
    Based on the following query from a user, please generate a detailed answer based on the context,
    focusing on which are the top 3 hotels based on the query. You should respond as if you are a travel agent and are conversing with the user.

    ###########
    query:
    "{query}"

    ########

    context:
    "{context}"
    #####

    Return the answer in Markdown format, with each hotel name highlighted.
    """

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.7,
        messages=messages
    )

    generated_text = response.choices[0].message['content'].strip()

    return generated_text

def render_markdown(md_text):
    html = markdown.markdown(md_text)
    display(HTML(html))

In [ ]:
query = "A good hotel with a great view on New York"
response = generate_answer(query)
render_markdown(response)

In [ ]:
import gradio as gr
def gradio_interface(query):
    response = generate_answer(query)
    return response

In [ ]:
iface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs="markdown",
    title="Top 3 Hotels Finder",
    description="Enter a query to find the best 3 hotels based on your preferences."
)

In [ ]:
iface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://59e7f8a495255b5109.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
